In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchinfo import summary
import joblib
torch.manual_seed(42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
validation_data=pd.read_csv('./assets/processed dataset/validation dataset.csv')
test_data=pd.read_csv('./assets/processed dataset/test dataset.csv')
train_data=pd.read_csv('./assets/processed dataset/train dataset.csv')
scaler =joblib.load('./assets/scaler.gz')

In [ ]:
df1=test_data
input_test=scaler.transform(df1)
input_test[0]

In [ ]:
seq_length=180
#Getting x and y for testing
X_test=[]
Y_test=[]

for i in range(seq_length,input_test.shape[0]):
    #Input features from 0 to 59
    X_test.append(input_test[i-seq_length:i])
    #T2M at 60
    Y_test.append(input_test[i])

# converting into numpy arrays
X_test,Y_test=np.array(X_test),np.array(Y_test)
Y_test=Y_test.reshape(-1,4)
print(X_test.shape,Y_test.shape)

X_test = Variable(torch.Tensor(X_test))
Y_test = Variable(torch.Tensor(Y_test))
print(X_test.shape,Y_test.shape)

In [ ]:
#single lstm Network definition
class NETWORK_SINGLE_LSTM(nn.Module):

    def __init__(self,num_classes,input_size,hidden_size,num_layers):
        super(NETWORK_SINGLE_LSTM,self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm=nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True).to(device)
        self.fc=nn.Linear(hidden_size,num_classes)
    
    def forward(self,x):
        h_0=torch.randn(self.num_layers, x.size(0), self.hidden_size).requires_grad_().to(device)
        c_0=torch.randn(self.num_layers, x.size(0), self.hidden_size).requires_grad_().to(device)
        output_lstm, (h_n,c_n) = self.lstm(x, (h_0, c_0))
        input_fc=output_lstm[:,-1,:]
        output_fc=self.fc(input_fc).to(device)
        return output_fc

In [ ]:
input_size=4
hidden_size=2
num_layers=1
num_classes=4

model=NETWORK_SINGLE_LSTM(num_classes,input_size, hidden_size,num_layers)
model.load_state_dict(torch.load('./assets/trained models state/LSTM_epochs_500_lr_0.01_hidden_2_layers_1.pt'))
model.eval()


In [ ]:
#Testing model
Y_pred=model(X_test.to(device))
Y_pred=Y_pred.detach().numpy()

In [ ]:
#MSE
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(Y_test,Y_pred)
mse

In [ ]:
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(Y_test,Y_pred)
mae

In [ ]:
#MAE
from sklearn.metrics import mean_absolute_error
mae_ps=mean_absolute_error(Y_test[0].flatten(),Y_pred[0].flatten())
print("PS MAE: %1.5f mm/hr"%mae_ps)
mae_qv2m=mean_absolute_error(Y_test[2].flatten(),Y_pred[2].flatten())
print("QV2M MAE: %1.5f gm/kg"%mae_qv2m)
mae_t2m=mean_absolute_error(Y_test[3].flatten(),Y_pred[3].flatten())
print("T2M MAE: %1.5f celsius"%mae_t2m)
mae_ws10m=mean_absolute_error(Y_test[1].flatten(),Y_pred[1].flatten())
print("WS10M MAE: %1.5f m/s"%mae_ws10m)

In [ ]:
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
mase=mean_absolute_scaled_error(Y_test,Y_pred,y_train=Y_test)
mase

In [ ]:
import torchvision
from torchview import draw_graph
with torch.no_grad():
    model_graph = draw_graph(model, input_size=(1,180,7), expand_nested=True)
model_graph.visual_graph